# Template Pipeline
Normalisation&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**-**  
Imputation 1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**Median**  
Outlier Detection&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**Z Score**  
Imputation 2&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**Median**  
Feature Selection&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**Implicit to model**  
Model&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**Lasso Regression**


## Imports

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# General
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# ML
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import QuantileTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.feature_selection import RFE

import xgboost as xgb

# Custom
import sys,os
%cd /content/fuck/My Drive/ETHZ/Autumn2020/AML/Git/AML/Task_1/Solution
sys.path.append('/content/fuck/My Drive/ETHZ/Autumn2020/AML/Git/AML/Task_1/Solution')
# sys.path.append('/home/jovyan/work/AML/Task_1/Solution') # I would like a cleaner solution but works for now
import Components.data_fetching as data_fetching

# # CAREFUL:
# # If you make changes to a custom module, you have to reload it, i.e rerun this cell
import importlib
importlib.reload(data_fetching)

[Errno 2] No such file or directory: '/content/fuck/My Drive/ETHZ/Autumn2020/AML/Git/AML/Task_1/Solution'
/home/jovyan


<module 'Components.data_fetching' from '/home/jovyan/work/AML/Task_1/Solution/Components/data_fetching.py'>

## Pipeline Optimisatoin

In [ ]:
X,y = data_fetching.get_train_data()
y = np.ravel(y)
x_test_final = data_fetching.get_test_data()

In [ ]:
nrm = QuantileTransformer(output_distribution='normal')
imp = IterativeImputer(missing_values=np.nan, max_iter=10, initial_strategy='median' ,random_state=0)
# imp = SimpleImputer(strategy='mean')
rfe = RFE(Ridge())
boost = xgb.XGBRegressor(objective='reg:squarederror')

pipe = Pipeline(steps=[('normalise', nrm),
                        ('impute', imp),
                        ('feature_select', rfe),
                        ('regress', boost)])

# Normalisation
n_quants = [50]
# Imputer
n_nearest_features = [10]
# Feature Selection
n_features = [50]
# Model
max_depth = [5,6]
learning_rate = [0.05, 0.1]
n_estimators = [200,300]

parameters = dict(normalise__n_quantiles=n_quants,
                    # impute__n_nearest_features=n_nearest_features,
                    feature_select__n_features_to_select=n_features,
                    regress__learning_rate=learning_rate,
                    regress__n_estimators=n_estimators)

clf = GridSearchCV(pipe, parameters)
clf.fit(X, y)

# View The Best Parameters
print('n_quantiles=', clf.best_estimator_.get_params()['normalise__n_quantiles'])
print('n_nearest_features=', clf.best_estimator_.get_params()['impute__n_nearest_features'])
print('n_features_to_select=', clf.best_estimator_.get_params()['feature_select__n_features_to_select'])
print('learning_rate=', clf.best_estimator_.get_params()['regress__learning_rate'])
print('n_estimators=', clf.best_estimator_.get_params()['regress__n_estimators'])
best_n_quantiles= clf.best_estimator_.get_params()['normalise__n_quantiles']
best_n_nearest_features= clf.best_estimator_.get_params()['impute__n_nearest_features']
best_n_features_to_select= clf.best_estimator_.get_params()['feature_select__n_features_to_select']
best_learning_rate= clf.best_estimator_.get_params()['regress__learning_rate']
best_n_estimators= clf.best_estimator_.get_params()['regress__n_estimators']

# Output
# n_quantiles= 50
# n_features_to_select= 50
# learning_rate= 0.05
# n_estimators= 200

## Final Prediction

In [ ]:
y_pred = clf.predict(x_test_final)
plt.hist(y_pred)
print("Train Score:", r2_score(y, clf.predict(X)))

y_pred_pd = pd.DataFrame(data=y_pred, columns=["y"])
y_pred_pd.to_csv('../Predictions/XGBoost_Optimised.csv', index_label='id')